In [ ]:
import pandas as pd

class AgingValores:
    def __init__(self, Name: str, Extension: str):
        self.archive = Name
        self.ext = f'.{Extension}'
        self.df = ''

    def LerArquivo(self, separator=';', encoding='latin-1'):
        if self.ext.upper() == '.CSV':
            self.df = pd.read_csv(self.archive+self.ext, encoding=encoding, sep=separator)
        elif self.ext.upper() == '.TXT':
            try:
                self.df = pd.read_csv(self.archive+self.ext, encoding=encoding, sep=separator)
            except Exception:
                print('Verifique o separador')
        elif self.ext.upper() == '.XLSX':
            self.df = pd.read_excel(self.archive+self.ext)
        else:
            print('Verifique se o arquivo existe no diretório ou se a extensão está correta!')
        return self.df

    def ManterColunas(self, *columns):
        list_columns = list(columns)
        self.df.columns = self.df.columns.str.strip()
        self.df = self.df[list_columns]
        return self.df
    
    def DeletarColunas(self, columns = None, startswith = None):
        self.df.columns = self.df.columns.str.strip()
        if type(columns) != list:
            print('Passe as colunas que serão deletadas em formato de lista!')
        else:
            list_columns = list(columns)
            self.df.drop(list_columns, axis=1, inplace=True)
        if startswith != None:
            self.df = (self.df.loc[:, ~self.df.columns.str.startswith(
                startswith)] if True in self.df.columns.str.startswith(startswith) else self.df)
        return self.df

    def RenomearColunas(self, ColumnsName: list, NewColumnsName: list):
        if type(ColumnsName) != list or type(NewColumnsName) != list:
            print('Passe os parâmetros em formato de lista []')
        elif len(ColumnsName) != len(NewColumnsName):
            print(
                'Quantidade de colunas incorreta, verifique se você colocou a mesma quantidade de colunas em ambas opções e tente novamente!')
        else:
            self.df.columns = self.df.columns.str.strip()
            for i, column in enumerate(ColumnsName):
                self.df.rename(columns={ColumnsName[i]: NewColumnsName[i]}, inplace=True)
            return self.df
    
    def TipoDeCompra(self, BuyType: str):
        self.df['Tipo_Compra'] = BuyType
        return self.df

    def CriarRangeValores(self, limits: list, labels: list, NumberField: str):

        self.df[NumberField] = self.df[NumberField].astype(str)
        self.df[NumberField] = self.df[NumberField].str.replace('.', '')
        self.df[NumberField] = self.df[NumberField].str.replace(',', '.')
        self.df[NumberField] = self.df[NumberField].astype(float)

        self.df["Range_valores"] = pd.cut(
            self.df[NumberField], bins=limits, labels=labels, include_lowest=True)
        return self.df

    def MostrarArquivo(self):
        return self.df

In [ ]:
df_indiretos = AgingValores('indiretos', 'xlsx')
df_diretos = AgingValores('pedidos-completos', 'txt')
df_diretos.LerArquivo(separator='|')
df_indiretos.LerArquivo()

In [ ]:
df_diretos.RenomearColunas(['Montante'], ['Valor bruto'])
df_indiretos.ManterColunas('Valor bruto')
df_diretos.ManterColunas('Valor bruto')

In [ ]:
limites = [0, 1000, 2000, 5000, 10000, 100000, 500000, float('inf')]

# define os rótulos dos intervalos
rotulos = ['R$ 0.00 - R$ 1.000', 'R$ 1.000 - R$ 2.000', 'R$ 2.000 - R$ 5.000', 'R$ 5.000 - R$ 10.000', 'R$ 10.000 - R$ 100.000',
           'R$ 100.000 - R$ 500.000', 'acima de R$ 500.000']
df_diretos.CriarRangeValores(labels=rotulos, limits=limites,
                             NumberField='Valor bruto')
df_indiretos.CriarRangeValores(labels=rotulos, limits=limites,
                               NumberField='Valor bruto')
df_diretos.TipoDeCompra('Diretos')
df_indiretos.TipoDeCompra('Indiretos')
df_diretos = df_diretos.MostrarArquivo()
df_indiretos = df_indiretos.MostrarArquivo()

In [ ]:
df_final = df_diretos.append(df_indiretos)
df_final

In [418]:
resumo = df_final.groupby("Range_valores").agg(
    {"Valor bruto": ["count", "sum"]})
resumo.columns = ["Quantidade", "Valor Total"]
resumo = resumo.reindex(rotulos)
resumo

,Quantidade,Valor Total
Range_valores,,
R$ 0.00 - R$ 1.000,930673,1.844449e+08
R$ 1.000 - R$ 2.000,68993,9.846301e+07
R$ 2.000 - R$ 5.000,65040,2.055698e+08
R$ 5.000 - R$ 10.000,35581,2.574311e+08
R$ 10.000 - R$ 100.000,48480,1.455305e+09
R$ 100.000 - R$ 500.000,10670,2.277467e+09
acima de R$ 500.000,11896,1.230603e+20
